This code has been lifted from the [PyTorch examples](https://github.com/pytorch/examples/blob/main/mnist/main.py).

## Setup

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from hessian_eigenthings import compute_hessian_eigenthings
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

In [2]:
SEED = 42
BATCH_SIZE = 64
TEST_BATCH_SIZE = 1000
LR = 1.0
GAMMA = 0.7
EPOCHS = 5
SAVE_MODEL = False

## Model

In [3]:
class Net(nn.Module):
    def __init__(self, hidden_dim=128):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, 10)

    def forward(self, x):
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

## Utils

In [4]:
def get_num_params(model):
    return sum(p.numel() for p in model.parameters())

## Basin Broadness
Calculated using equation that can be found in [this LessWrong post](https://www.lesswrong.com/posts/QPqztHpToij2nx7ET/hessian-and-basin-volume).

In [5]:
def get_hessian_spectrum(model):
    pass

def get_c(init_std):
    pass

def get_basin_volume(model, _lambda, init_std):
    pass

## Train & Test Functions

In [6]:
def train(model, device, train_loader, optimizer, epoch, dry_run=False, log_interval=100):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            if dry_run:
                break

In [7]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

## Execution

In [8]:
use_cuda = torch.cuda.is_available()
use_mps = torch.backends.mps.is_available()

torch.manual_seed(SEED) # for reproducibility

if use_cuda:
    device = torch.device("cuda")
elif use_mps:
    device = torch.device("mps")
else:
    device = torch.device("cpu")

train_kwargs = {'batch_size': BATCH_SIZE}
test_kwargs = {'batch_size': TEST_BATCH_SIZE}
if use_cuda:
    cuda_kwargs = {'num_workers': 1,
                    'pin_memory': True,
                    'shuffle': True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)

transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])
dataset1 = datasets.MNIST('../data', train=True, download=True,
                    transform=transform)
dataset2 = datasets.MNIST('../data', train=False,
                    transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

model = Net(hidden_dim=24).to(device)
optimizer = optim.Adadelta(model.parameters(), lr=LR)

scheduler = StepLR(optimizer, step_size=1, gamma=GAMMA)
for epoch in range(1, EPOCHS + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

if SAVE_MODEL:
    torch.save(model.state_dict(), "mnist_cnn.pt")

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.351011
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.279978
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.257797
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.316638
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.157406
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.312547
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.164181
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.327729
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.201208
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.241340

Test set: Average loss: -6.5851, Accuracy: 9404/10000 (94%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.090885
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.137226
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.084006
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.268485
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.116145
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.213458
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.100226
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.276355
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.153515


In [11]:
num_eigenthings = 10
loss = torch.nn.CrossEntropyLoss()
eigenvals, eigenvecs = compute_hessian_eigenthings(model.cpu(), test_loader, loss, num_eigenthings, use_gpu=False)

 [=============================================================>...]  Step: 504ms | Tot: 9s669ms | power iter error: 0.00 20/20 
 [=============================================================>...]  Step: 501ms | Tot: 9s653ms | power iter error: 0.00 20/20 
 [=============================================================>...]  Step: 506ms | Tot: 9s651ms | power iter error: 0.00 20/20 


KeyboardInterrupt: 